![Toronto AI](https://i.imgur.com/diILtDP.png)

# Neural Networks

A trained neural net can be thought of as a learned mapping.

Here are some examples of mappings that a neural net could learn:

*   Mapping English to French
*   Mapping pictures to text descriptions
*   Mapping live sensor data from a reusable rocket to control commands that land it
*   Mapping random vectors into images of flowers

In essence, we use neural nets to map one distribution of data onto another.

Here's an an example where I trained a neural net to map random vectors onto the space of flower photos, using a Generative Adversarial Network:

![](https://i.imgur.com/SaT9OEM.png)

# Tensors

* Tensors are multidimensional arrays.
* They are like boxes of data, that we use to contain our data, or the weights of our model.
* Tensors are used extensively in TensorFlow to represent:
  * 0-D - scalars
  * 1-D - vectors, text
  * 2-D - matrices, tables of data
  * 3-D - batches of matrices, a cube of data, e.g. an image, a monochrome video
  * 4-D - convolution kernels, a colour video
  * 5-D - batches of colour video
  * 6-D - 3D vector fields
  * 7-D - layered 3D vector fields (e.g. gravity and electromagnetism layers)
  * 8-D - batches of layered 3D vector fields 
  * 9-D - batches of layered 3D vector fields evolving through time
  * keep going...

* GPU memory is expensive, so Tensors are most commonly 4-D or less.

* It helps to visualize a Tensor as a Rubiks Cube - each cell holds a piece of scalar data (like a weight, a piece of input data, or a label).  For higher dimensional Tensors, think of each cell as holding a Tensor instead of a scalar.
![](https://i.imgur.com/KyOQVX9.png)

In [1]:
import tensorflow as tf
import os

DIRECTORY = os.path.join("~", "ai")

# Synthetic Dataset


In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random, math
from matplotlib import animation, rc
from IPython.display import HTML

plt.rcParams['image.cmap'] = 'viridis'

np.random.seed(20180118)

BATCH_SIZE = 16
DIM = 64
TWO_PI = 2.*math.pi
MIN_CIRCLE_WIDTH = int(DIM/10)
NUM_CIRCLE_DOTS = 500
CIRCLE_WIDTH = 3
DARKNESS = .5

# We'll use this to create our data set.
def add_circles(data):
    radius = int((random.random() * (DIM/2 - MIN_CIRCLE_WIDTH)) + MIN_CIRCLE_WIDTH)
    xpos = random.random()*DIM
    ypos = random.random()*DIM
    
    draw_circle(data, xpos, ypos, radius, DARKNESS, CIRCLE_WIDTH)

    return [xpos, ypos, radius+CIRCLE_WIDTH/2]


def draw_circle(data, xpos, ypos, radius, darkness, width):
    for i in range(NUM_CIRCLE_DOTS):
        for r in range(radius, radius+width):
            rad = TWO_PI * i/NUM_CIRCLE_DOTS
            x = int(round(r*math.cos(rad)+xpos))
            y = int(round(-r*math.sin(rad)+ypos))
            if x >= 0 and x < DIM and y >= 0 and y < DIM:
                data[x,y] = data[x,y] - darkness


            
# Create random noise and draw circles in it
def create_dataset_row():
    data = np.random.random((DIM, DIM)).astype(np.float32)
    label = add_circles(data)
    label = np.array(label).astype(np.float32)
    
    return (data, label)
    

def create_dataset(rows):
    
    labels  = []
    samples = []
    for i in range(rows):
        data, label = create_dataset_row()
        labels.append(label)
        samples.append(data)
    return (np.array(samples).astype(np.float32), np.array(labels).astype(np.float32))

        

# Create an animation so we can see our data set
dataset = create_dataset(BATCH_SIZE)
data   = dataset[0]
labels = dataset[1]


def plot_dataset(data):
    fig, ax = plt.subplots()

    def init():
        ax.cla()
        return ()

    def animate(i):
        ax.imshow(data[i%len(data)])
        return ()

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=BATCH_SIZE, interval=700, blit=True)
    plt.close(fig)
    fig.set_size_inches(10, 10, True)
    return HTML(anim.to_jshtml())


    
plot_dataset(data)

# GOALS

* Predict the radius of the circle
* Predict the center of the circle

We are going to create a neural net to solve this problem for us, since it would be very difficult to solve this with conventional code.

![](https://i.imgur.com/o2qIsu4.png)

# Convolutional Layers


* a convolutional layer is used to extract 'feature maps' from the previous layer.
* feature maps are called 'channels' in TensorFlow.
* Channels are familiar.  Images often have three feature maps: the Red, Green and Blue channels.

* You can stack convolution layers.  Learn basic features from an image in one layer, then learn features of those features in the next layer.

* Convolutional layers can be employed to learn dozens of feature maps off the data.


![](https://upload.wikimedia.org/wikipedia/commons/6/63/Typical_cnn.png)
_Image credits: Aphex34 (Wikimedia Commons)_





### Convolutional layers learn feature maps, one for each filter.
* Each filter acts like a small window through which it can read a specific feature from each channel of the input.
* Each filter passes across the entire image and at each position combines the features detected in the input channels into an output channel.
* A convolutional layer with many filters will have an output channel for each filter, holding the results.


In [3]:
# docs: https://www.tensorflow.org/api_docs/python/tf/layers/conv2d

def Convolution(layer, name, num_filters, size=3, activation=tf.nn.relu): 

    with tf.variable_scope("detector", reuse=tf.AUTO_REUSE):
        
        layer = tf.layers.conv2d( 
              name=name, 
              inputs=layer,
              kernel_size=size,
              filters=num_filters,
              padding='same',
              activation=activation # the activation function used: nonlinearity gets applied
        )
        return layer
    


### Activation functions
* To add 'depth' to our convolutional layer (i.e. the depth in deep learning), we need to add a non-linearity to the output, called an activation function.
* We'll use a Leaky ReLu (Rectified Linear Unit) - it's nonlinear and simple.  ELUs are a great choice as well.

<table>
    <tr>
        <th>tf.nn.leaky_relu</th><td><img src="https://i.imgur.com/KxYFRIL.png =100x100" alt="Drawing" style="width: 200px;"/>
        </td>
    </tr>
    <tr><th>Other activation functions</th><td>
        <p><code>tf.nn.relu</code> <i>(dense, convolution)</i></p>
            <p><code>tf.nn.elu</code> <i>(dense, convolution)</i></p>
            <p><code>tf.nn.softplus</code> <i>(dense, convolution)</i></p>
            <p><code>tf.nn.sigmoid</code> <i>(0 to 1 classifier)</i></p>
            <p><code>tf.nn.tanh</code> <i>(LSTM, -1 to 1 classifier)</i></p>
    </td>
    </tr>
</table>

<table>
        <th>other functions</th><td><img src="https://4.bp.blogspot.com/-4puZ_ZMyoLE/WPk_rwqhKnI/AAAAAAAAAjU/vrNE_Uv54yMLOd_3E83PvgpByf019ufZwCLcB/s1600/ActivationFunctions.png =200x200" alt="Drawing" style="width: 400px;"/>
        </td>
</table>

</br>
Dense means dense layers or fully connected layers
</br>
For image recognitions, relu and leaky relu are good options.
</br>
Further reading on activation functions:
* https://en.wikipedia.org/wiki/Rectifier_(neural_networks) - Overview of Retifiers in neural nets
* https://arxiv.org/abs/1505.00853 - Empirical Evaluation of Rectified Activations in Convolutional Network
* https://arxiv.org/abs/1709.06247 - Training Better CNNs Requires to Rethink ReLU

# Pooling

* Shrink a layer
* Using a stride of two will halve the width and height.
* The pool_size should be at least as large as the strides

In [4]:



def AvgPooling(layer): 
    return tf.layers.average_pooling2d(layer, pool_size=(2,2), strides=(2,2))

def MaxPooling(layer): 
    return tf.layers.max_pooling2d(layer, pool_size=(2,2), strides=(2,2))

# Creating our graph

## Subgraph: The convolutional layers

* We are stacking our convolutional layers, so that later layers detect features on lower layers.
* Higher layers learn higher level features from lower layers that learn lower level features.


In [5]:

def convolutional_layers(batch):
    
    layer = tf.reshape(batch, [BATCH_SIZE, DIM, DIM, 1])  # We need a channel dimension, adding one here.
    
    with tf.variable_scope("convolutional"):
        
        FS = 32 # maximum features
        K  = 7  # kernel size
        
        layer = Convolution(layer, "A1", num_filters=FS, size=K)
        layer = Convolution(layer, "A2", num_filters=FS, size=K)
        layer = Convolution(layer, "A3", num_filters=FS, size=K)

        layer = AvgPooling(Convolution(layer, "A4", num_filters=FS, size=K))
        layer = AvgPooling(Convolution(layer, "A5", num_filters=FS, size=K))
        layer = AvgPooling(Convolution(layer, "A6", num_filters=FS, size=K))

        
#         layer =  AvgPooling(layer)
#         layer = Convolution(layer, "L4", num_filters=FS, size=5)
        
#         layer =  AvgPooling(layer)
#         layer = Convolution(layer, "L5", num_filters=FS, size=3)
        
        return layer
    

After our image goes through the convolutional layer, we still need to do some inference through the dense layers.

## Subgraph: The fully connected dense layers

* These layers are used to convert the tensor that was output from the convolutional layers down into a prediction.
* In our case, we want 3 outputs
  
  * The X and Y coordinate of the center of the circle
  * The radius of the circle

* Here we're dividing the channels into thirds, and we have attached one subnet of fully connected layers to each output.  
* We are asking the neural net that the sum of the outputs of each subnet is the corresponding prediction (for x, y, radius)


In [6]:

def dense_layers(layer): 
    
    layer = tf.reshape(layer, [BATCH_SIZE, -1])    # Make it a table, one row for each example in the batch.
    # -1 specifies taking all of the data
    
    x_guess = add_subnet(layer, 0)  # Let's create a subnet for predicting the X coordinate,
    y_guess = add_subnet(layer, 1)  # A subnet for the y coordinate,
    r_guess = add_subnet(layer, 2)  # and, a subnet for predicting the radius
    
    y_guess2 = add_subnet(layer, 1)  # A subnet for the y coordinate,
    
    return x_guess, (y_guess+y_guess2)/2., r_guess




def add_subnet(layer, i):
    ROW_DIM = 1
    a_third  = int(layer.shape[ROW_DIM].value / 3)
    
    # taking a third of the data
    layer = tf.slice(layer, [0, i*a_third], [BATCH_SIZE, a_third])
    
    
    layer = tf.layers.dense(layer, 32,  activation=tf.nn.elu) # 32 layers of neurons to pass through
    layer = tf.layers.dense(layer, 32,  activation=tf.nn.elu) # then into the next 32 layers of neurons to pass through
    layer = tf.layers.dense(layer, 32,  activation=tf.nn.elu)
    # layer size: 32x32x32

    # Return the sum of the last layer as our prediction
    return tf.reduce_sum(layer, axis=[1])



# Objective function (a.k.a. the loss function)

* The objective function is what the system attempts to minimize
* The most important thing to remember is that the loss function needs to be differentiable with a minimum value at your goal.
* Convex functions are easy to minimize.

### Common objectives
* Minimizing the difference of squares (a.k.a. mean squared error)

* Minimizing the log loss - this is useful in classification tasks when dealing with probabilities.


In [7]:
# keep this differentiable: smooth (not bumpy) so derivatives can be easily computed

def loss_function(x, y, r, labels):

    
    actual_x, actual_y, actual_r = labels[:,0], labels[:,1], labels[:,2]
    
    
    # Define an error between the predicted x, y and radius, and their actual values
    # We will later ask the network to minimize this error through gradient descent
    
    x_error = tf.pow(x - actual_x, 2.)
    # x_error = tf.square(x - actual_x, 2.)
    y_error = tf.pow(y - actual_y, 2.)
    r_error = tf.pow(r - actual_r, 2.)

    
    # Let's combine these individual loss metrics into a combined loss function
    # Note: these are vectors - with separate losses defined for each element in the batch     
    # reduce the difference between predictive value and actual value with gradient descent
    loss = x_error + y_error + r_error

    # Now lets create some metrics that we can use to evaluate our progress
    avg_distance_from_actual_center = tf.reduce_mean(tf.sqrt(x_error + y_error))
    avg_radius_error = tf.reduce_mean(r_error)
    
    tf.summary.scalar("avg_distance_from_actual_center", avg_distance_from_actual_center)
    tf.summary.scalar("avg_radius_error",                avg_radius_error)
    
    return {
        "combined" : loss,
        "center_error_px" : avg_distance_from_actual_center,
        "radius_error_px" : avg_radius_error
    }
    


# Optimizer

There are many choices for optimizers.
For most applications, the Adam optimizer will give you good flexibility and fast training.

## Adam

* Currently the most popular optimizer

* The adam optimizer is a gradient descent optimization alogirthm that adds two things:
  * First, it adds momentum to each weight of your model to help it descend.
  * Second, it slows down weights proportionally to how much they are oscillating (variance)
  
* Both of the effects of the Adam optimizer have an exponential decay built in.  These are parameters to the optimizer.
  * alpha - The learning rate.  Typical values are 0.0003 to to 0.000003
  * beta1 - The decay rate of the momentum term.  Typical values are 0.5 to 0.9.
  * beta2 - The decay rate of the variance term.  Typical values are 0.9 to 0.999



Further reading: 
* https://arxiv.org/abs/1412.6980 Adam: A Method for Stochastic Optimization
* http://ruder.io/optimizing-gradient-descent/ An overview of gradient descent optimization algorithms


In [8]:
ALPHA = 0.0001
BETA1 = 0.5
BETA2 = 0.9


def add_optimizer(loss):
    
    opt      = tf.train.AdamOptimizer(learning_rate = ALPHA, beta1=BETA1, beta2=BETA2)
    train_op = opt.minimize(loss=loss) # ask the optimizer to minimize the loss
    return train_op
    

# Connecting the model pieces together


In [9]:


def create_model(samples, labels):

    x, y, r = dense_layers( convolutional_layers( samples ) ) # connect to dense layers

    losses = loss_function(x, y, r, labels)

    training_op = add_optimizer(losses["combined"])

    return {
        "training_op"   : training_op,
        "losses"          : losses,
        "samples"       : samples,
        "prediction"    : (x, y, r),
        "actual"        : labels
    }

# Preparing a Training run

In [10]:
import itertools

tf.reset_default_graph()

def row_generator():
    for i in itertools.count(1):
        yield create_dataset_row()

# creating a data pipeline
def create_input_pipeline():
    # use the generator to generate dynamic dataset
    dataset = tf.data.Dataset.from_generator(row_generator, (tf.float32, tf.float32))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat()
        
    pipeline_iterator = dataset.make_initializable_iterator()
    
    return pipeline_iterator


latest_results = None
def run_session(session, run_name, num_iterations):
    global latest_results # make this global so we don't have to finish the training to see the result!
    np.random.seed(20180118)
    ema_radius = 500
    ema_center = 500
    
    UPDATE_TENSORBOARD_PERIOD = 1

    with tf.device("/device:CPU:0"): # use GPU if you have it
            # a file writer to write session
        fw = tf.summary.FileWriter(DIRECTORY + run_name, graph=session.graph, flush_secs=2)

    
        # 
        pipeline = create_input_pipeline()

        sample_batch, labels_batch = pipeline.get_next()

        # 
        model = create_model(sample_batch, labels_batch)

        # 
        tf.global_variables_initializer().run()

        # initial the pipeline, create a session
        session.run(pipeline.initializer)

        # for each iteration, we run the sample batch to get a new batch
        for step in range(num_iterations):
            
            new_batch = session.run(sample_batch)
            
            # pieces being trained
            latest_results = session.run({
                "losses"     : model["losses"],
                "train_op"   : model["training_op"],
                "samples"    : model["samples"],
                "prediction" : model["prediction"],
                "actual"     : model["actual"],
                "summary"    : tf.summary.merge_all()
            })
            
            # report radius error in pixels 
            # use exponential moving average that keeps 90% of previous value, add 10% of new, super simple to implement
            stats = latest_results["losses"]
            ema_radius = ema_radius * .9  + .1 * stats["radius_error_px"]
            ema_center = ema_center * .9  + .1 * stats["center_error_px"]
            
            
            if step % UPDATE_TENSORBOARD_PERIOD == 0:
                print("step: {step}, EMA radius error: {ema_radius:.0f}px. EMA center error: {ema_center:.0f}px".format(step=step, ema_radius=ema_radius, ema_center=ema_center))
                fw.add_summary(latest_results["summary"], step)
        
        return latest_results

        


# Start Training!

In [11]:
tf.reset_default_graph()
session = tf.Session()

with session:
    
    latest_results = run_session(session, "100steps", 100) #"1000steps", 1000)

print("Complete!")
# ResNet w 8 filters: step: 100, ema radius error: 71.01px. Avg center error: 16.13px

step: 0, EMA radius error: 500px. EMA center error: 455px
step: 1, EMA radius error: 500px. EMA center error: 415px
step: 2, EMA radius error: 485px. EMA center error: 379px
step: 3, EMA radius error: 476px. EMA center error: 345px
step: 4, EMA radius error: 461px. EMA center error: 315px
step: 5, EMA radius error: 458px. EMA center error: 288px
step: 6, EMA radius error: 459px. EMA center error: 264px
step: 7, EMA radius error: 455px. EMA center error: 242px
step: 8, EMA radius error: 443px. EMA center error: 221px
step: 9, EMA radius error: 419px. EMA center error: 203px
step: 10, EMA radius error: 388px. EMA center error: 187px
step: 11, EMA radius error: 359px. EMA center error: 170px
step: 12, EMA radius error: 331px. EMA center error: 156px
step: 13, EMA radius error: 314px. EMA center error: 143px
step: 14, EMA radius error: 297px. EMA center error: 131px
step: 15, EMA radius error: 273px. EMA center error: 120px
step: 16, EMA radius error: 252px. EMA center error: 110px
step: 1

# Visualizing the Results!

In [12]:

sample_batch = np.asarray(latest_results["samples"])
prediction   = latest_results["prediction"]
actual       = latest_results["actual"].transpose()

# Extract and round
xp, yp, rp      = [np.rint(p).astype(np.int32) for p in prediction]
xa, ya, ra      = [np.rint(a).astype(np.int32) for a in actual]

result_display = []


# Add the prediction and the actual to the dataset
for i, sample in enumerate(sample_batch):
    draw_circle(sample, xa[i], ya[i], ra[i],  DARKNESS, CIRCLE_WIDTH)
    draw_circle(sample, xp[i], yp[i], rp[i], -DARKNESS, CIRCLE_WIDTH)  # The prediction will appear bright
    result_display.append(sample)


plot_dataset(np.array(result_display))



Based on our model result, it is not doing very well. There is room for improvement!

For potential cryto-grant, fork the project, PR with your model with 500 runs of training. If you can improve it, do it!

# Thank you!

That was a lot of info!
Thank you for your time and hopefully this will help you on your path.

For your troubles, here's a flower:
![](https://i.imgur.com/i6wAoY9.png)
<p style="text-align:center">AI generated flower, orchestration by Dave MacDonald</p>

# Connect with the Toronto AI community

<img src="https://cdn.worldvectorlogo.com/logos/slack-1.svg" style="display:inline;width: 30px;padding-right:1em;"/> Come join us on our Slack Channel!  (Here is the <a href="https://join.slack.com/t/toronto-ai/shared_invite/enQtMjE5NTM5MzY3NTU0LTQ0ZDIyM2ZlZDYwMmRjY2I2NTEyMjZjYzJkNzljZTI1ZWRiMDkzYjUyZjRkMTc5ZDM0OGJmZjdmNzM5NDM5Zjk">Invite link</a> if you haven't signed up yet)

<img src="https://cdn.worldvectorlogo.com/logos/meetup.svg" style="display:inline;width: 30px;padding-right:1em;"/> Come to our next event - join our [Meetup Group](https://www.meetup.com/Toronto-AI/)

<img src="https://i.imgur.com/KGSBbUe.png" style="display:inline;width:30px;padding-right:1em;3"/> NEO tips <span style='font-size:.75em'>AdLG9AyRtCMSeAy98rmkkos7uFU6i7fLgd</span>


Toronto AI website: [http://torontoai.org/](http://torontoai.org/)<br>
We're also on Facebook and Twitter
<br><br>
Thank you, see you soon!


## Additional Resources

* [Deep Learning Book](http://www.deeplearningbook.org/)